In [170]:
from llama_index.core import StorageContext, load_index_from_storage
from llama_index.llms.openai import OpenAI
import openai
import os
from dotenv import load_dotenv

load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")
os.environ["OPENAI_API_KEY"] = openai.api_key

In [171]:
class Answer_agent:
    def __init__(self, history):
        self.client = openai.OpenAI()
        self.history = history
        self.description = [
            {"role": "system", "content": "you are an expert in Elden Ring, answer the user's prompt"},
        ]

    def answer(self):
        response = self.client.chat.completions.create(
            model="gpt-4o",
            messages=self.history + self.description,
            max_tokens=1000,
            n=1,
            top_p=0.95,
            frequency_penalty=0.2,
            temperature=0.7
        )
        return response.choices[0].message.content

In [172]:
class RAG_agent:
    def __init__(self, index_path):
        storage_context = StorageContext.from_defaults(persist_dir=index_path)
        index = load_index_from_storage(storage_context)
        llm = OpenAI(
            model="gpt-4o",
            temperature=0.7,
            top_p=0.95,
            frequency_penalty=0.2,
            num_outputs=1000,
        )
        self.query_engine = index.as_query_engine(response_mode="refine", similarity_top_k=5, verbose = True, llm = llm)

    def query(self, query):
        return self.query_engine.query(query)


In [173]:
class Query_agent:
    # This agent takes user prompt and refine to a query for specific RAG agent
    def __init__(self, description, history):
        self.client = openai.OpenAI()
        self.history = history
        self.description = [
            {"role": "system", "content": description},
        ]

    def create_query(self):
        response = self.client.chat.completions.create(
            model="gpt-4o",
            messages=self.history + self.description,
            max_tokens=1000,
            n=1,
            top_p=0.95,
            frequency_penalty=0.2,
            temperature=0.7
        )
        return response.choices[0].message.content
        

In [174]:
class Response_agent:
    # This agent takes the history and create the final response to user
    def __init__(self, description, history):
        self.client = openai.OpenAI()
        self.messages = history
        self.description = [
            {"role": "system", "content": description},
        ]

    def respond(self):
        response = self.client.chat.completions.create(
            model="gpt-4o",
            messages=self.messages + self.description,
            max_tokens=1000,
            n=1,
            top_p=0.95,
            frequency_penalty=0.2,
            temperature=0.7
        )
        return response.choices[0].message.content

In [175]:
KEYWORD = "ADDITIONAL_QUERY_REQUIRED"

In [176]:
class EldenGuideSystem:
    # The agent system that takes user prompt and return the final response
    def __init__(self, strategy_agent, gameinfo_agent, 
                 strategy_query_description, 
                 gameinfo_query_description, 
                 loop_response_description, 
                 final_response_description):
        self.messages = []
        self.strategy_agent = strategy_agent
        self.gameinfo_agent = gameinfo_agent
        self.answer_agent = Answer_agent(self.messages)
        self.strategy_query_agent = Query_agent(strategy_query_description, self.messages)
        self.gameinfo_query_agent = Query_agent(gameinfo_query_description, self.messages)
        self.loop_response_agent = Response_agent(loop_response_description, self.messages)
        self.final_response_agent = Response_agent(final_response_description, self.messages)

    def push_assistant_message(self, message):
        self.messages.append({
            "role": "assistant",
            "content": message
        })

    def clear_messages(self):
        self.messages = []

    def run(self, prompt):
        count = 0
        self.messages.append({"role": "user", "content": prompt})
        answer = self.answer_agent.answer()
        self.push_assistant_message(answer)
        while count < 3:
            print(f"Querying... {count}")
            strategy_query = self.strategy_query_agent.create_query()
            strategy_response = self.strategy_agent.query(strategy_query)
            self.push_assistant_message("Strategy query result: " + str(strategy_response))
            gameinfo_query = self.gameinfo_query_agent.create_query()
            gameinfo_response = self.gameinfo_agent.query(gameinfo_query)
            self.push_assistant_message("In-game data query result: " + str(gameinfo_response))
            loop_response = self.loop_response_agent.respond()
            if KEYWORD in loop_response:
                count += 1
            else:
                break

        if count >= 3:
            final_response = self.final_response_agent.respond()
        else:
            final_response = loop_response
        return final_response


In [177]:
def load_prompt(filename):
    with open(f'agent_descriptions/{filename}.txt', 'r') as file:
        return file.read().strip()

STRATEGY_QUERY_DESCRIPTION = load_prompt('strategy_agent')
GAMEINFO_QUERY_DESCRIPTION = load_prompt('gameinfo_agent')
FINAL_RESPONSE_DESCRIPTION = load_prompt('final_agent')
LOOP_RESPONSE_DESCRIPTION = load_prompt('loop_agent')

In [94]:
# index paths
strategy_path = "index/strategy"
gameinfo_path = "index/game"
altogether_path = "index/altogether"

In [35]:
strategy_agent = RAG_agent(strategy_path)
gameinfo_agent = RAG_agent(gameinfo_path)

In [186]:
prompt = "I'm currently playing as a Level 35 Vagabond, focusing mainly on Strength and Vigor. I've been using the Greatsword, but I feel like my damage output isn't cutting it against bosses like Rennala, Queen of the Full Moon. I also keep getting one-shotted by some of her spells. Any tips on how to build my character more effectively or strategies to defeat her? Should I be investing in Magic resistance or maybe switch to a different weapon? Any help would be greatly appreciated!"

In [187]:
system = EldenGuideSystem(strategy_agent, gameinfo_agent,
                          STRATEGY_QUERY_DESCRIPTION,
                          GAMEINFO_QUERY_DESCRIPTION,
                          LOOP_RESPONSE_DESCRIPTION,
                          FINAL_RESPONSE_DESCRIPTION)
response = system.run(prompt)
print(response)

Querying... 0
To effectively deal with Rennala, Queen of the Full Moon, as a Level 35 Vagabond focusing on Strength and Vigor, here are some consolidated tips based on your current situation:

### Build and Equipment Adjustments:

1. **Armor and Resistance:**
   - Equip armor with high Magic Defense to mitigate her powerful spells. The **Carian Knight Set** is ideal if you have it.
   - Consider using a shield with high Magic resistance. The **Carian Knight's Shield** is a good option and can be found in Raya Lucaria Academy in the graveyard area.

2. **Stat Allocation:**
   - Continue prioritizing Vigor to increase your HP and survivability.
   - Consider allocating points into Endurance for better stamina management and to handle heavier armor or shields for additional protection.

3. **Weapon Upgrades:**
   - Ensure your Greatsword is upgraded as much as possible using Smithing Stones to maximize damage output.
   - If available, experiment with faster weapons or those with magic sc

In [188]:
for message in system.messages:
    print(message)


{'role': 'user', 'content': "I'm currently playing as a Level 35 Vagabond, focusing mainly on Strength and Vigor. I've been using the Greatsword, but I feel like my damage output isn't cutting it against bosses like Rennala, Queen of the Full Moon. I also keep getting one-shotted by some of her spells. Any tips on how to build my character more effectively or strategies to defeat her? Should I be investing in Magic resistance or maybe switch to a different weapon? Any help would be greatly appreciated!"}
{'role': 'assistant', 'content': "To tackle Rennala, Queen of the Full Moon, with your Level 35 Vagabond focusing on Strength and Vigor, here are some tips and strategies to improve your chances of success:\n\n### Build Adjustments:\n\n1. **Armor and Resistance:**\n   - Since you're getting one-shotted by her spells, consider boosting your Magic Resistance. Equip armor with high Magic Defense. The **Carian Knight Set** is a solid choice if you have it.\n   - Use a shield with good Magi

In [183]:
feedback = "I want specific build, including talismans, armors, etc."
response = system.run(feedback)
print(response)


Querying... 0
Here's a detailed build for your level 100 katana character to effectively take on Malenia, focusing on bleed damage and agility:

### Stats Allocation:
- **Vigor:** 40-50 (for survivability)
- **Mind:** 15-20 (enough to use skills and spirit summons)
- **Endurance:** 20-25 (to manage stamina for dodging and attacking)
- **Strength:** Meet the minimum requirement for your chosen katana
- **Dexterity:** 50+ (primary damage stat for katanas)
- **Intelligence/Faith/Arcane:** Allocate only if your weapon scales with these stats

### Weapon:
- **Rivers of Blood:** This katana is excellent for its bleed potential and the Corpse Piler skill, which deals heavy damage and builds up bleed quickly.
- **Moonveil:** If you prefer a mix of magic and physical attacks, Moonveil's Transient Moonlight skill can be very powerful.

### Ashes of War:
- Use the katana's innate skill if you're wielding Rivers of Blood or Moonveil. Otherwise, consider **Bloody Slash** or **Seppuku** for addition

In [185]:
for message in system.messages:
    print(message)


{'role': 'user', 'content': "I'm level 100 now, I want to use a kanata to defeat Malenia, recommend me a build."}
{'role': 'assistant', 'content': "Congratulations on reaching level 100! Using a katana to defeat Malenia is a great choice, as katanas offer a good balance of speed, reach, and bleed damage potential. Here's a build recommendation to help you take her down:\n\n### Stats Allocation:\n- **Vigor:** 40-50 (High vigor is essential to survive Malenia's powerful attacks)\n- **Mind:** 15-20 (Ensure you have enough FP for Ashes of War and Spirit Summons)\n- **Endurance:** 20-30 (Enough stamina to dodge and attack effectively)\n- **Strength:** Base requirement for the katana you choose\n- **Dexterity:** 50+ (Primary damage stat for katanas)\n- **Intelligence/Faith/Arcane:** Only invest if your chosen katana or Ash of War scales with these\n\n### Weapon:\n- **Katana Options:**\n  - **Uchigatana:** A solid choice with good bleed buildup.\n  - **Moonveil:** Excellent for its mix of phy

In [182]:

client = openai.OpenAI()

description = [
    {"role": "user", "content": "I'm level 100 now, I want to use a kanata to defeat Malenia, recommend me a build."},
]

response = client.chat.completions.create(
    model="gpt-4o",
    messages=description,
    max_tokens=1000,
    n=1,
    top_p=0.95,
    frequency_penalty=0.2,
    temperature=0.7
)

print(response.choices[0].message.content)

Congratulations on reaching level 100! Facing Malenia in Elden Ring with a katana can be both challenging and rewarding. Here is a suggested build to help you defeat her:

### Attributes:
1. **Vigor**: Aim for around 40-50. Malenia hits hard, so having a good health pool is essential.
2. **Mind**: Around 20-25 should suffice, especially if you plan to use skills that consume FP.
3. **Endurance**: 25-30 to ensure you have enough stamina for dodging and attacking.
4. **Strength**: Minimum required for your katana of choice (typically around 12-18).
5. **Dexterity**: Invest most of your points here, aiming for at least 40-50, since katanas scale well with Dexterity.
6. **Intelligence/Faith/Arcane**: Allocate points according to any specific Ashes of War or incantations you plan to use.

### Weapon:
- **Katana Choice**: The Uchigatana, Nagakiba, or Moonveil are popular choices. Moonveil is particularly potent due to its unique skill, Transient Moonlight, which can deal significant damage a